<a href="https://colab.research.google.com/github/Rodeffs/Year4_Programming/blob/master/Big_Data/03_lab/hadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Checking java version

In [15]:
!java --version

openjdk 17.0.16 2025-07-15
OpenJDK Runtime Environment (build 17.0.16+8-Ubuntu-0ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 17.0.16+8-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)


Installing Hadoop

In [22]:
!wget https://dlcdn.apache.org/hadoop/common/hadoop-3.4.2/hadoop-3.4.2.tar.gz

--2025-12-02 08:38:59--  https://dlcdn.apache.org/hadoop/common/hadoop-3.4.2/hadoop-3.4.2.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1065831750 (1016M) [application/x-gzip]
Saving to: ‘hadoop-3.4.2.tar.gz’

hadoop-3.4.2.tar.gz 100%[===================>]   1016M  59.9MB/s    in 11s     

2025-12-02 08:39:11 (89.0 MB/s) - ‘hadoop-3.4.2.tar.gz’ saved [1065831750/1065831750]



In [30]:
!tar -xf hadoop-3.4.2.tar.gz

tar: hadoop: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


Installing the dataset

In [37]:
!curl -L https://www.kaggle.com/api/v1/datasets/download/beta3logic/3m-academic-papers-titles-and-abstracts -o dataset.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1409M  100 1409M    0     0  93.1M      0  0:00:15  0:00:15 --:--:--  111M


In [40]:
!unzip dataset.zip

Archive:  dataset.zip
  inflating: cleaned_papers.csv      


Setting the variables

In [19]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["JRE_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64/jre"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.4.2"
os.environ["PATH"] += ":$JAVA_HOME/bin:$JRE_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin"